In [2]:
from collections import defaultdict
import requests
from time import sleep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta, time
from urllib.error import HTTPError
import statistics

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#
cur_working_directiory = os.getcwd()
# Change 
os.chdir("../../../Data/")


CSVfiles = []
for file in os.listdir("./"):
  if file.endswith(".csv"):
    CSVfiles.append(file)
print(CSVfiles)


unixTimeConverter = lambda x : (datetime.fromtimestamp(x)).strftime('%Y-%m-%d %H:%M:%S')
unixTimeConverterNoDateJustTime = lambda x: (datetime.fromtimestamp(x) - timedelta(hours=4)).time()


allDfs = []


for x in CSVfiles:

  dfTemp = pd.read_csv(x)
  dfTemp['estTime']  = dfTemp['time'].apply(unixTimeConverter)
  dfTemp['timeOnly'] = dfTemp['time'].apply(unixTimeConverterNoDateJustTime)

  dfTemp['candleHeight']  = dfTemp.apply( lambda row:    max(row['open'],  row['close']) - min(row['open'],  row['close']), axis = 1 )
  allDfs.append(dfTemp)

df = allDfs[0]
#This is for generating if profit
for i in range(1, 15):
  df['P/L' +str(i) +'Short_']  = ( df['close'] - df['close'].shift(-i) )
  df['P/L' +str(i) +'Long_']  =  (  df['close'].shift(-i) - df['close']     )
  df['P/L' +str(i) +'Short']  = ( df['close'] - df['close'].shift(i) )
  df['P/L' +str(i) +'Long']  =  (  df['close'].shift(i) - df['close']     )

df = df.dropna()


os.chdir(cur_working_directiory)

df.head()





['OANDA EURUSD, 5(2).csv', 'BATS AAPL, 30.csv', 'OANDA EURUSD, 240(1).csv', 'OANDA EURUSD, 60(3).csv', 'OANDA EURUSD, 30(4).csv']


,time,open,high,low,close,Volume,Volume MA,estTime,timeOnly,candleHeight,...,P/L12Short,P/L12Long,P/L13Short_,P/L13Long_,P/L13Short,P/L13Long,P/L14Short_,P/L14Long_,P/L14Short,P/L14Long
14,1681399800,1.10544,1.10549,1.10522,1.10528,290,545.80,2023-04-13 11:30:00,07:30:00,0.00016,...,-0.00052,0.00052,-0.00004,0.00004,-0.00008,0.00008,0.00021,-0.00021,0.00052,-0.00052
15,1681400100,1.10528,1.10590,1.10528,1.10572,343,523.55,2023-04-13 11:35:00,07:35:00,0.00044,...,0.00074,-0.00074,0.00065,-0.00065,-0.00008,0.00008,0.00066,-0.00066,0.00036,-0.00036
16,1681400400,1.10572,1.10594,1.10572,1.10580,350,503.60,2023-04-13 11:40:00,07:40:00,0.00008,...,0.00092,-0.00092,0.00074,-0.00074,0.00082,-0.00082,0.00042,-0.00042,0.00000,0.00000
17,1681400700,1.10580,1.10616,1.10574,1.10600,355,489.65,2023-04-13 11:45:00,07:45:00,0.00020,...,0.00126,-0.00126,0.00062,-0.00062,0.00112,-0.00112,0.00042,-0.00042,0.00102,-0.00102
18,1681401000,1.10600,1.10602,1.10520,1.10522,392,477.50,2023-04-13 11:50:00,07:50:00,0.00078,...,0.00053,-0.00053,-0.00036,0.00036,0.00048,-0.00048,-0.00040,0.00040,0.00034,-0.00034


In [3]:
d_counts_average_vol            = defaultdict(list)
d_counts_average_highLow_diff   = defaultdict(list)


closeOpenDeltaList = []
closeOpenDeltaList_ = []
openLowDeltaList = []
highLowDeltaList = []

for index, row in df.iterrows():
  closeOpenDeltaList.append(abs(row['close'] - row['open']))
  closeOpenDeltaList_.append(row['close'] - row['open'])
  openLowDeltaList.append(abs(row['open'] - row['low']))
  highLowDeltaList.append(row['high'] - row['low'])


df1 = pd.DataFrame({'closeOpenDelta': closeOpenDeltaList, 'CloseOpenDelta_': closeOpenDeltaList_, 'openLowDelta': openLowDeltaList, 'highLowDelta': highLowDeltaList})

# df2 = pd.concat([df, df1])

df2 = pd.concat([df.reset_index(drop=True), df1.reset_index(drop=True)], axis=1)
print("mnath should add up")
print(df.shape)
print(df1.shape)
print(df2.shape)
df= df2

for index, row in df.iterrows():
  #put volumes in d
  time_  = str(row['timeOnly'])
  volume = row['Volume']
  d_counts_average_vol[str(time_)].append(volume)


  #Collecting, using a dictionary of lists to collect the price deltas for each time
  price_delta = row['closeOpenDelta']
  d_counts_average_highLow_diff[str(time_)].append(price_delta)

df.head()

###############################################
##THIS use to be seperate cells, but now I just combined them
##
################################################
import statistics

std_values_with_time_as_keys = {}
mean_values_with_time_as_keys = {}

#for volume
std_values_with_time_as_keys_volume = {}
mean_values_with_time_as_keys_volume = {}

for k , v in d_counts_average_highLow_diff.items():

  # Calculate mean and standard deviation of the sample for price delta
  mean = statistics.mean(v)
  std_dev = statistics.stdev(v)
  std_values_with_time_as_keys[k] = std_dev
  mean_values_with_time_as_keys[k] = mean

for k , v in d_counts_average_vol.items():
  # Calculate mean and standard deviation of the sample of volume
  mean__ = statistics.mean(v)
  std_dev__ = statistics.stdev(v)
  std_values_with_time_as_keys_volume[k] = std_dev__
  mean_values_with_time_as_keys_volume[k] = mean__


z_score_list = []
z_score_volume_list = []

for index, row in df.iterrows():
  time_  = str(row['timeOnly'])
  mean_ = mean_values_with_time_as_keys[time_]
  std_ = std_values_with_time_as_keys[time_]

  cur_val = row['closeOpenDelta']
  z_score_list.append(( cur_val - mean)/ std_)
  # row['z-score'] = (row['closeOpenDelta'] - mean)/ std_
  
  mean_ = mean_values_with_time_as_keys_volume[time_]
  std_ = std_values_with_time_as_keys_volume[time_]
  df.loc[index, 'z-score-volume'] = (row['Volume'] - mean)/ std_

df['z-score']= z_score_list 
# df['z-score-volume']= 


df.head(1)

mnath should add up
(16751, 66)
(16751, 4)
(16751, 70)


,time,open,high,low,close,Volume,Volume MA,estTime,timeOnly,candleHeight,...,P/L14Short_,P/L14Long_,P/L14Short,P/L14Long,closeOpenDelta,CloseOpenDelta_,openLowDelta,highLowDelta,z-score-volume,z-score
0,1681399800,1.10544,1.10549,1.10522,1.10528,290,545.8,2023-04-13 11:30:00,07:30:00,0.00016,...,0.00021,-0.00021,0.00052,-0.00052,0.00016,-0.00016,0.00022,0.00027,1.914775,-0.123742


In [4]:
""" Create a lookup table for mean and std for c++ bot """

out_dictionary = {}

for k, v in std_values_with_time_as_keys.items():
    out_dictionary[k] = [mean_values_with_time_as_keys[k], v]
with open("EUR_USD_M5_price_delta.json", "w") as outfile:
    json.dump(out_dictionary, outfile, indent=4)

Index(['time', 'open', 'high', 'low', 'close', 'Volume', 'Volume MA',
       'estTime', 'timeOnly', 'candleHeight', 'P/L1Short_', 'P/L1Long_',
       'P/L1Short', 'P/L1Long', 'P/L2Short_', 'P/L2Long_', 'P/L2Short',
       'P/L2Long', 'P/L3Short_', 'P/L3Long_', 'P/L3Short', 'P/L3Long',
       'P/L4Short_', 'P/L4Long_', 'P/L4Short', 'P/L4Long', 'P/L5Short_',
       'P/L5Long_', 'P/L5Short', 'P/L5Long', 'P/L6Short_', 'P/L6Long_',
       'P/L6Short', 'P/L6Long', 'P/L7Short_', 'P/L7Long_', 'P/L7Short',
       'P/L7Long', 'P/L8Short_', 'P/L8Long_', 'P/L8Short', 'P/L8Long',
       'P/L9Short_', 'P/L9Long_', 'P/L9Short', 'P/L9Long', 'P/L10Short_',
       'P/L10Long_', 'P/L10Short', 'P/L10Long', 'P/L11Short_', 'P/L11Long_',
       'P/L11Short', 'P/L11Long', 'P/L12Short_', 'P/L12Long_', 'P/L12Short',
       'P/L12Long', 'P/L13Short_', 'P/L13Long_', 'P/L13Short', 'P/L13Long',
       'P/L14Short_', 'P/L14Long_', 'P/L14Short', 'P/L14Long',
       'closeOpenDelta', 'CloseOpenDelta_', 'openLowDelta',